Thinking1：既然内容相似度计算简单，能频繁更新，为什么还需要协同过滤算法呢？
   - 协同过滤可以挖掘用户之间或物品之间潜在的相关性，能够动态调整权重。
   
Thinking2：你需要推荐系统么？哪些情况下不需要推荐系统？
   - 需要，购物、就餐、游玩都需要推荐系统。出行不需要。
   
Thinking3: 如果给一个视频打标签，视频中有音乐作为背景音乐，采用了NLP方式对内容自动打标签，可能存在什么问题？
   - NLP可能会识别背景音乐中的歌词，并当作视频内容的一部分。

Action1: 针对mnist数据集进行分类，采用CART决策树（工具使用sklearn中的CART），可以参考课程代码 mnist_lr.py

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV

digits = load_digits()
data = digits.data

print(data.shape)

print(digits.images[0])
print(digits.target[0])

# plt.gray()
# plt.title('Hand Written Digits')
# plt.imshow(digits.images[0])
# plt.show()

ss = preprocessing.StandardScaler()
data = ss.fit_transform(data)

train_x, test_x, train_y, test_y = train_test_split(data, digits.target, test_size=0.25, random_state=33)

cart = DecisionTreeClassifier()
cart.fit(train_x, train_y)
predict_y = cart.predict(test_x)
print('CART accuracy:%0.4lf' % accuracy_score(predict_y, test_y))

(1797, 64)
[[ 0.  0.  5. 13.  9.  1.  0.  0.]
 [ 0.  0. 13. 15. 10. 15.  5.  0.]
 [ 0.  3. 15.  2.  0. 11.  8.  0.]
 [ 0.  4. 12.  0.  0.  8.  8.  0.]
 [ 0.  5.  8.  0.  0.  9.  8.  0.]
 [ 0.  4. 11.  0.  1. 12.  7.  0.]
 [ 0.  2. 14.  5. 10. 12.  0.  0.]
 [ 0.  0.  6. 13. 10.  0.  0.  0.]]
0
CART accuracy:0.8600


Action2: 员工离职预测 

In Class Competition 

我们有员工的各种统计信息，以及该员工是否已经离职，统计的信息包括了（工资、出差、工作环境满意度、工作投入度、是否加班、是否升职、工资提升比例等） 
现在需要你来通过训练数据得出 员工离职预测，并给出你在测试集上的预测结果 

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score
# load data
train_df = pd.read_csv('train.csv')
train_df['Attrition'] = train_df['Attrition'].map({'Yes':1, 'No':0})
test_df = pd.read_csv('test.csv')
comb = pd.concat([train_df, test_df])
# preprocess
cat_cols = [col for col in comb.select_dtypes(object).columns if col not in ['user_id','Attrition']]
for col in cat_cols:
    tmp = pd.get_dummies(comb[col],prefix = col)
    comb = comb.drop([col],axis=1)
    comb = pd.concat([comb,tmp],axis=1)

features = [col for col in comb.columns if col not in ['user_id','Attrition']]
comb_x = comb[features]
# nomalization
mm = MinMaxScaler()
scaled_features = mm.fit_transform(comb_x)
comb_x = pd.DataFrame(scaled_features, index=comb_x.index, columns=comb_x.columns)

data_x = comb_x[:train_df.shape[0]]
data_y = train_df['Attrition']
pred_x = comb_x[train_df.shape[0]:]

train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.25, random_state=33)

lr = LogisticRegression()
lr.fit(train_x, train_y)
predict_y = lr.predict(test_x)
print('LR accuracy:%0.4lf' % accuracy_score(predict_y, test_y))

pred_result=pd.DataFrame(test_df[['user_id']])
pred_result['Attrition']=lr.predict_proba(pred_x)[:,1]
print(len(pred_result))
pred_result.to_csv('baseline_lr.csv',index=False) 

FileNotFoundError: [Errno 2] File train.csv does not exist: 'train.csv'